# 🎲 Tabular Data Generation with Diffusion Models

**Generate High-Quality Synthetic Tabular Data**

This notebook demonstrates how to generate synthetic tabular data using diffusion models with explainability.

## 📋 What You'll Learn:
1. ✅ Load and explore tabular datasets
2. ✅ Train a specialized tabular diffusion model
3. ✅ Generate synthetic data that matches real distributions
4. ✅ Visualize and compare real vs synthetic data
5. ✅ Evaluate data quality

## 🎯 Use Cases:
- **Privacy-preserving data sharing** (medical records, financial data)
- **Data augmentation** for imbalanced datasets
- **Testing and development** with realistic synthetic data
- **Research** without access to real sensitive data

---

**⚠️ Note**: This notebook works best with GPU but can run on CPU (slower)

In [ ]:
# 🚀 GOOGLE COLAB SETUP - Run this cell first if using Colab!
import os
import sys
import subprocess

# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("🌐 Running in Google Colab")
except:
    IN_COLAB = False
    print("💻 Running locally")

if IN_COLAB:
    print("\n" + "="*70)
    print("🔧 Setting up environment for Google Colab...")
    print("="*70)
    
    # Remove old clone if exists
    if os.path.exists('/content/Data-generation'):
        print("\n🗑️  Removing old clone...")
        subprocess.run(['rm', '-rf', '/content/Data-generation'], check=True)
    
    # Clone repository
    print("\n📥 Cloning repository...")
    subprocess.run(['git', 'clone', 'https://github.com/ayansh1729/Data-generation.git'], 
                   cwd='/content', check=True)
    print("   ✅ Repository cloned")
    
    # Change to project directory
    os.chdir('/content/Data-generation')
    print(f"   📂 Working directory: {os.getcwd()}")
    
    # Install dependencies
    print("\n📦 Installing dependencies...")
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-r', 'requirements.txt'], check=True)
    print("   ✅ Dependencies installed")
    
    # Install package
    print("\n⚙️  Installing project...")
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-e', '.'], check=True)
    print("   ✅ Project installed")
    
    # Add to path
    sys.path.insert(0, '/content/Data-generation')
    
    # Check GPU
    import torch
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"\n🎮 GPU Available: {gpu_name}")
        print(f"   Memory: {gpu_memory:.1f} GB")
    else:
        print("\n⚠️  No GPU detected. Training will be slower.")
        print("   💡 Enable GPU: Runtime → Change runtime type → GPU")
    
    print("\n" + "="*70)
    print("✅ Setup complete!")
    print("="*70)
else:
    print("ℹ️  Not in Colab - using local environment")

## 📚 Import Libraries


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer, load_wine, load_diabetes
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Project imports
from src.models.tabular_diffusion import TabularDiffusionModel

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Using device: {device}")
print(f"🐍 PyTorch version: {torch.__version__}")


## 📊 Load and Explore Dataset

We'll use the **Breast Cancer Wisconsin** dataset - a classic medical dataset with 30 features.


In [ ]:
# Load dataset
data = load_breast_cancer()
X = data.data
y = data.target
feature_names = data.feature_names

print(f"📊 Dataset: Breast Cancer Wisconsin")
print(f"   Samples: {X.shape[0]}")
print(f"   Features: {X.shape[1]}")
print(f"   Classes: {len(np.unique(y))} (Malignant: {(y==0).sum()}, Benign: {(y==1).sum()})")

# Create DataFrame
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y

# Display first few rows
print("\n📋 First few samples:")
df.head()


In [ ]:
# Statistical summary
print("📈 Statistical Summary:")
df.describe()


In [ ]:
# Visualize feature distributions
fig, axes = plt.subplots(3, 3, figsize=(16, 12))
axes = axes.flatten()

for i in range(9):
    axes[i].hist(X[:, i], bins=40, alpha=0.7, color='steelblue', edgecolor='black')
    axes[i].set_title(f"{feature_names[i]}", fontsize=11, fontweight='bold')
    axes[i].set_xlabel('Value', fontsize=9)
    axes[i].set_ylabel('Frequency', fontsize=9)
    axes[i].grid(alpha=0.3)

plt.tight_layout()
plt.suptitle('📊 Feature Distributions (Real Data)', y=1.002, fontsize=15, fontweight='bold')
plt.show()

print("✅ First 9 features visualized")


In [ ]:
# PCA Visualization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(10, 7))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis', 
                     alpha=0.6, edgecolors='k', s=50)
plt.colorbar(scatter, label='Class (0=Malignant, 1=Benign)')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)', fontsize=12)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)', fontsize=12)
plt.title('🔍 Real Data - PCA Visualization', fontweight='bold', fontsize=14)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"✅ PCA explains {(pca.explained_variance_ratio_[0] + pca.explained_variance_ratio_[1])*100:.1f}% of variance")


## 🔧 Prepare Data for Training


In [ ]:
# Normalize the data
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Create simple dataset class
class TabularDataset(Dataset):
    def __init__(self, data):
        self.data = torch.FloatTensor(data)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

# Create dataset and dataloader
dataset = TabularDataset(X_normalized)
dataloader = DataLoader(
    dataset,
    batch_size=64,
    shuffle=True,
    drop_last=True
)

print(f"✅ Data prepared")
print(f"   Dataset size: {len(dataset)}")
print(f"   Batch size: 64")
print(f"   Number of batches: {len(dataloader)}")
print(f"   Normalized range: [{X_normalized.min():.2f}, {X_normalized.max():.2f}]")


## 🏗️ Create Tabular Diffusion Model

We use a specialized MLP-based architecture (not U-Net) optimized for tabular data.


In [ ]:
# Create Tabular Diffusion Model
input_dim = X.shape[1]

model = TabularDiffusionModel(
    input_dim=input_dim,
    hidden_dims=(256, 512, 512, 256),
    time_emb_dim=128,
    dropout=0.1,
    use_batch_norm=True
).to(device)

# Create DDPM wrapper for training and sampling
class TabularDDPM(nn.Module):
    def __init__(self, model, timesteps=1000):
        super().__init__()
        self.model = model
        self.timesteps = timesteps
        
        # Cosine noise schedule (better for tabular data)
        steps = timesteps + 1
        x = torch.linspace(0, timesteps, steps)
        alphas_cumprod = torch.cos(((x / timesteps) + 0.008) / 1.008 * torch.pi * 0.5) ** 2
        alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
        betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
        betas = torch.clip(betas, 0.0001, 0.9999)
        
        alphas = 1.0 - betas
        alphas_cumprod = torch.cumprod(alphas, dim=0)
        
        self.register_buffer('betas', betas)
        self.register_buffer('alphas', alphas)
        self.register_buffer('alphas_cumprod', alphas_cumprod)
        self.register_buffer('sqrt_alphas_cumprod', torch.sqrt(alphas_cumprod))
        self.register_buffer('sqrt_one_minus_alphas_cumprod', torch.sqrt(1.0 - alphas_cumprod))
    
    def forward(self, x):
        """Training forward pass."""
        batch_size = x.shape[0]
        device = x.device
        
        # Random timesteps
        t = torch.randint(0, self.timesteps, (batch_size,), device=device)
        noise = torch.randn_like(x)
        
        # Add noise to data
        sqrt_alpha = self.sqrt_alphas_cumprod[t].unsqueeze(-1)
        sqrt_one_minus_alpha = self.sqrt_one_minus_alphas_cumprod[t].unsqueeze(-1)
        x_noisy = sqrt_alpha * x + sqrt_one_minus_alpha * noise
        
        # Predict noise
        predicted = self.model(x_noisy, t)
        loss = nn.functional.mse_loss(predicted, noise)
        
        return loss
    
    @torch.no_grad()
    def sample(self, num_samples, dim):
        """Generate samples via reverse diffusion."""
        device = next(self.parameters()).device
        x = torch.randn(num_samples, dim, device=device)
        
        for t in tqdm(reversed(range(self.timesteps)), desc='🎲 Generating', total=self.timesteps):
            t_batch = torch.full((num_samples,), t, device=device, dtype=torch.long)
            predicted_noise = self.model(x, t_batch)
            
            alpha_t = self.alphas[t]
            alpha_cumprod_t = self.alphas_cumprod[t]
            beta_t = self.betas[t]
            
            if t > 0:
                noise = torch.randn_like(x)
            else:
                noise = torch.zeros_like(x)
            
            x = (x - beta_t / torch.sqrt(1 - alpha_cumprod_t) * predicted_noise) / torch.sqrt(alpha_t)
            x = x + torch.sqrt(beta_t) * noise
        
        return x

ddpm = TabularDDPM(model, timesteps=1000).to(device)

num_params = sum(p.numel() for p in ddpm.parameters())
print(f"\\n✅ Model created successfully!")
print(f"   Input dimension: {input_dim}")
print(f"   Hidden dimensions: (256, 512, 512, 256)")
print(f"   Time embedding: 128")
print(f"   Total parameters: {num_params:,}")
print(f"   Timesteps: 1000")
print(f"   Noise schedule: Cosine")
